<a href="https://colab.research.google.com/github/buzhangjiuzhou/DLfinal/blob/master/bert_algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
# 导入包
# transformer提供了一些训练好的模型，可以很方便的使用。
!pip install transformers
import tensorflow as tf
import pandas as pd
import os
import tqdm
# 使用分类的模型，增加了一个head用于分类。
from transformers import BertTokenizer, TFBertForSequenceClassification
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
% matplotlib inline

tf.__version__

'2.2.0'

In [42]:
# 导入现成的模型和分词器
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', tokenize_chinese_chars=True)
model = TFBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=7)
model.summary()
model.config

Some weights of the model checkpoint at bert-base-uncased were not used when initializing TFBertForSequenceClassification: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier', 'dropout_75']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  109482240 
_________________________________________________________________
dropout_75 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  5383      
Total params: 109,487,623
Trainable params: 109,487,623
Non-trainable params: 0
_________________________________________________________________


BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "type_vocab_size": 2,
  "vocab_size": 30522
}


In [32]:
# 数据链接，可以在 https://course.fast.ai/datasets 找到。
# agnews数据集，类别，标题，描述。
train_csv = 'https://raw.githubusercontent.com/buzhangjiuzhou/dataset/master/algorithm/train.csv'
test_csv = 'https://raw.githubusercontent.com/buzhangjiuzhou/dataset/master/algorithm/vali.csv'

In [33]:
# 下载数据，并指定此时数据集的目录
train_file = tf.keras.utils.get_file(origin=train_csv,fname='train.csv')
test_file = tf.keras.utils.get_file(origin=test_csv,fname='test.csv')

In [34]:
# 读取数据
# base_dir = ''
train = pd.read_csv(train_file,engine='python',encoding = 'gbk')
test = pd.read_csv(test_file,engine='python',encoding = 'gbk')
print(train)
print(len(train))
print(train.head())
print(len(test))
print(test.head())

        id                                            content  ...  parallel  angle
0      613  E和F分别是边AB和AC中点，D是BC边上一点，若角AED等于角AFD，请问四边形AEDF是...  ...         0      0
1      148  在三角形ADC中，角ADC等于90角，三角形DC全等BDH，那么BH与AC互相垂直吗？请说明理由．  ...         0      0
2      782  （2019春玉溪期末）已知比OA垂直OB，点C在射线OB上，经过C点的直线DF平行OE，角B...  ...         1      0
3      679      （2018春拜泉县期末）已知比直线AB与CD被EF所截，角1等于角2，求证比AB平行CD．  ...         1      0
4      544  在平面直角坐标系中，A（2，1），B（4，负3），且以A，B，O，C为顶点的四边形为平行四边...  ...         0      0
...    ...                                                ...  ...       ...    ...
5286  3027  已知三角形ABC的三边长分别为23，32，30，与三角形ABC相似的三角形A′B′C′的最大...  ...         0      1
5287  3028  （2019春蔡甸区期末）已知AB平行CD，角ABE等于110角，角DCE等于36角，求角BE...  ...         0      0
5288  3029  三角形ADE相似三角形ABC，AD等于40，BD等于20，BC等于50，角A等于70角，角A...  ...         1      0
5289  3030  O为直线AC上一点，OB，OD，OE为射线，OD是角AOB的平分线，角BOE等于1除2角EO...  ...         0      0
5290  3030  O为直线AC上一点，OB，OD，OE为射线，OD是∠AOB的平分线，∠BOE=1/2∠EOC...  ...         0

In [51]:
train[:10]

,id,content,vertical,trian,triangle,rectangle,diamond,parallel,angle
0,613,E和F分别是边AB和AC中点，D是BC边上一点，若角AED等于角AFD，请问四边形AEDF是...,0,0,0,0,1,0,0
1,148,在三角形ADC中，角ADC等于90角，三角形DC全等BDH，那么BH与AC互相垂直吗？请说明理由．,0,1,0,0,0,0,0
2,782,（2019春玉溪期末）已知比OA垂直OB，点C在射线OB上，经过C点的直线DF平行OE，角B...,1,0,0,0,0,1,0
3,679,（2018春拜泉县期末）已知比直线AB与CD被EF所截，角1等于角2，求证比AB平行CD．,0,0,0,0,0,1,0
4,544,在平面直角坐标系中，A（2，1），B（4，负3），且以A，B，O，C为顶点的四边形为平行四边...,0,0,0,0,1,0,0
5,275,D是BC上一点，AB等于AD，BC等于DE．三角形ABC全等三角形ADE，求证比角CDE等于...,0,1,0,0,0,0,0
6,1419,在长方形ABCD中，角DAE等于角CBE等于45角，AD等于1，分别写出长度为有理数的线段与...,0,0,1,0,0,0,0
7,1127,在矩形ABCD中，AB等于5厘米，BC等于7厘米，点E从点A出发，沿AB方向以1厘米除s的速...,0,1,0,0,0,0,0
8,1071,已知已知AB垂直AC，角DAB等于角C，求角CDA的度数．,0,1,0,0,0,0,0
9,482,直线a、b被c所截，角1减角2等于11角，角3相加角4等于169角，求角1和角2的度数．[方...,0,0,0,0,0,0,1


In [35]:
columns = test.columns[2:]
train_labels = []
test_labels = []
for i in tqdm.notebook.trange(len(train)):
  for j in range(len(columns)):
    if train[columns[j]][i] == 1:
      train_labels.append(j)
      break
for i in tqdm.notebook.trange(len(test)):
  for j in range(len(columns)):
    if test[columns[j]][i] == 1:
      test_labels.append(j)
      break

In [49]:
# transformers自带的tokenizer中的encoder会把一段文本进行编码，然后增加上CLS和SEP，其中CLS的id是101，SEP的编码是102,PAD是0。
# 所以  a   dog   is  not   a   table
# [cls]  a   dog   is  not   a   table  [sep]
# 101   1037  3899  2003 2025  1037  2795   102
%pprint #让列表横过来，好看一些。
tokenizer.encode(text='E为平行四边形ABCD外一点，且AE垂直EC，BE垂直ED，平行四边形ABCD是矩形吗？请说明理由．', padding='max_length',max_length=512)

Pretty printing has been turned OFF


[101, 1041, 100, 1839, 1945, 1798, 100, 100, 5925, 2094, 1809, 1740, 100, 1989, 100, 29347, 100, 100, 14925, 1989, 2022, 100, 100, 3968, 1989, 1839, 1945, 1798, 100, 100, 5925, 2094, 100, 100, 100, 100, 1994, 100, 100, 1865, 100, 100, 1991, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [41]:
train.content[2369]

'E为平行四边形ABCD外一点，且AE垂直EC，BE垂直ED，平行四边形ABCD是矩形吗？请说明理由．'

In [50]:
# 预处理数据
# 按照上面的例子把数据集中的文本进行处理，并且得到对应的labels。
max_length = 64
train_ids = [tokenizer.encode(text=sent, padding='max_length', max_length=max_length, return_tensors="tf") for sent in tqdm.notebook.tqdm(train.content)]
test_ids = [tokenizer.encode(text=sent, padding='max_length',max_length=max_length, return_tensors="tf") for sent in tqdm.notebook.tqdm(test.content)]
train_labels = train[0].values - 1
test_labels = test[0].values - 1

ValueError: ignored

In [ ]:
# 把数据转成tensorflow张量
# train_ids是tf.Tensor组成得列表，所以用concat组合一下就行
train_ids = tf.concat(train_ids, 0)
# 把train_mask初始化为1，然后把train_ids等于0（PAD的部分）对应的值赋为0
train_mask = tf.ones(train_ids.shape)
train_mask = tf.where(tf.math.greater(train_ids, 0), train_mask, 0)
# labels本身是numpy数组，转为tf.Tensor
train_labels = tf.convert_to_tensor(train_labels)

# 测试集的处理同理test
test_ids = tf.concat(test_ids, 0)
test_mask = tf.ones(test_ids.shape)
test_mask = tf.where(tf.math.greater(test_ids, 0), test_mask, 0)
test_labels = tf.convert_to_tensor(test_labels)

print(train_ids[0])
print(train_mask[0])
print(train_labels[0])

In [ ]:
# 训练参数
epochs = 1
batch_size = 4
validation_rate = 0.1

In [ ]:
# 训练
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, clipnorm=1.0)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [ ]:
# 模型训练
model.fit(x=[train_ids, train_mask], 
     y=train_labels, 
     batch_size=batch_size, 
     epochs=epochs, 
     verbose=1, 
     callbacks=None,
     validation_split=validation_rate, 
     validation_data=None, 
     shuffle=True)

In [ ]:
# 模型测试
model.evaluate(x=[test_ids, test_mask],
        y=test_labels, 
        batch_size=4, 
        verbose=1)